#### Import Dependencies

In [2]:
import pandas as pd
import requests
from census import Census
from config import api_key
import pymongo

#### Census API Calls

In [3]:
#2019
c19 = Census(api_key, year=2019)
# See: https://github.com/datamade/census for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data19 = c19.acs5.get(("NAME", "B01003_001E",
                                      "B17001_002E",
                                      "B23025_005E",
                                      "B15003_002E",
                                      "B15003_017E",
                                      "B15003_021E",
                                      "B15003_022E",
                                      "B02001_002E",
                                      "B02001_003E",
                                      "B02001_004E",
                                      "B02001_005E",
                                      "B02001_006E",
                                      "B03001_003E",
                                      "B02001_008E"), {'for': 'state:*'})

# Convert to DataFrame
census_19 = pd.DataFrame(census_data19)

# Column Reordering
census_19 = census_19.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "PovertyCount",
                                      "B23025_005E": "UnemploymentCount",
                                      "B15003_002E": "NoCompletedEducation",
                                      "B15003_017E": "PopulationWithHighSchoolDiploma",
                                      "B15003_021E": "PopulationWithAssociatesDegree",
                                      "B15003_022E": "PopulationWithBachelorsDegree",
                                      "B02001_002E": "WhitePop",
                                      "B02001_003E": "BlackPop",
                                      "B02001_004E": "NativeAmericanPop",
                                      "B02001_005E": "AsianPop",
                                      "B02001_006E": "NativeHawaiianPop",
                                      "B03001_003E": "HispanicPop",
                                      "B02001_008E": "TwoOrMoreRacesPop",
                                      "NAME": "Name", "state": "State"})

# Add in Poverty Rate (Poverty Count / Population)
census_19["PovertyRate"] = 100 * \
    census_19["PovertyCount"].astype(
        int) / census_19["Population"].astype(int)
# Add in Unemployment Rate (Employment Count / Population)
census_19["UnemploymentRate"] = 100 * \
    census_19["UnemploymentCount"].astype(
        int) / census_19["Population"].astype(int)
# Final DataFrame
census_19 = census_19[["State", "Name", "Population", "PovertyCount", "PovertyRate", "UnemploymentRate",
                        "NoCompletedEducation", "PopulationWithHighSchoolDiploma",
                      "PopulationWithAssociatesDegree", "PopulationWithBachelorsDegree",
                      "WhitePop", "BlackPop", "NativeAmericanPop", "AsianPop", "NativeHawaiianPop",
                      "HispanicPop", "TwoOrMoreRacesPop"]]

census_19.head(10)

,State,Name,Population,PovertyCount,PovertyRate,UnemploymentRate,NoCompletedEducation,PopulationWithHighSchoolDiploma,PopulationWithAssociatesDegree,PopulationWithBachelorsDegree,WhitePop,BlackPop,NativeAmericanPop,AsianPop,NativeHawaiianPop,HispanicPop,TwoOrMoreRacesPop
0,01,Alabama,4876250.0,795989.0,16.323794,2.708946,40837.0,841716.0,282316.0,529178.0,3320247.0,1299048.0,25565.0,66270.0,2238.0,208626.0,92220.0
1,02,Alaska,737068.0,76933.0,10.437707,3.637114,3434.0,108797.0,41109.0,88058.0,476015.0,24205.0,109751.0,45920.0,9204.0,51870.0,60665.0
2,04,Arizona,7050299.0,1043764.0,14.804535,2.778676,71270.0,940168.0,408312.0,869452.0,5444453.0,317462.0,317414.0,233213.0,14458.0,2208663.0,263037.0
3,05,Arkansas,2999370.0,496260.0,16.545475,2.349860,23602.0,559816.0,143979.0,297250.0,2301044.0,459542.0,20434.0,45504.0,8733.0,224130.0,80537.0
4,06,California,39283497.0,5149742.0,13.109174,3.052765,713452.0,4813250.0,2073823.0,5603047.0,23453222.0,2274108.0,303998.0,5692423.0,155290.0,15327688.0,1922664.0
5,08,Colorado,5610349.0,565873.0,10.086235,2.341583,37709.0,667781.0,321903.0,981017.0,4712574.0,233647.0,54847.0,178147.0,8643.0,1208172.0,205724.0
6,10,Delaware,957248.0,109400.0,11.428595,2.766368,6505.0,187323.0,52636.0,124632.0,658237.0,212302.0,3729.0,37009.0,542.0,88364.0,27079.0
7,11,District of Columbia,692683.0,107140.0,15.467393,4.037489,5406.0,70065.0,14792.0,122380.0,285857.0,320811.0,2091.0,27592.0,376.0,76191.0,21445.0
8,09,Connecticut,3575074.0,344146.0,9.626262,3.257443,26484.0,588032.0,191964.0,541380.0,2714031.0,383416.0,9955.0,161257.0,1123.0,574240.0,119000.0
9,12,Florida,20901636.0,2870487.0,13.733313,2.686144,224991.0,3647514.0,1468744.0,2827938.0,15702256.0,3359031.0,59320.0,571276.0,12653.0,5346684.0,572021.0


In [5]:
#2018
c18 = Census(api_key, year=2018)
census_data18 = c18.acs5.get(("NAME", "B01003_001E",
                                      "B17001_002E",
                                      "B23025_005E",
                                      "B15003_002E",
                                      "B15003_017E",
                                      "B15003_021E",
                                      "B15003_022E",
                                      "B02001_002E",
                                      "B02001_003E",
                                      "B02001_004E",
                                      "B02001_005E",
                                      "B02001_006E",
                                      "B03001_003E",
                                      "B02001_008E"), {'for': 'state:*'})

# Convert to DataFrame
census_18 = pd.DataFrame(census_data18)

# Column Reordering
census_18 = census_18.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "PovertyCount",
                                      "B23025_005E": "UnemploymentCount",
                                      "B15003_002E": "NoCompletedEducation",
                                      "B15003_017E": "PopulationWithHighSchoolDiploma",
                                      "B15003_021E": "PopulationWithAssociatesDegree",
                                      "B15003_022E": "PopulationWithBachelorsDegree",
                                      "B02001_002E": "WhitePop",
                                      "B02001_003E": "BlackPop",
                                      "B02001_004E": "NativeAmericanPop",
                                      "B02001_005E": "AsianPop",
                                      "B02001_006E": "NativeHawaiianPop",
                                      "B03001_003E": "HispanicPop",
                                      "B02001_008E": "TwoOrMoreRacesPop",
                                      "NAME": "Name", "state": "State"})

# Add in Poverty Rate (Poverty Count / Population)
census_18["PovertyRate"] = 100 * \
    census_18["PovertyCount"].astype(
        int) / census_18["Population"].astype(int)
# Add in Unemployment Rate (Employment Count / Population)
census_18["UnemploymentRate"] = 100 * \
    census_18["UnemploymentCount"].astype(
        int) / census_18["Population"].astype(int)
# Final DataFrame
census_18 = census_18[["State", "Name", "Population", "PovertyCount", "PovertyRate", "UnemploymentRate",
                        "NoCompletedEducation", "PopulationWithHighSchoolDiploma",
                      "PopulationWithAssociatesDegree", "PopulationWithBachelorsDegree",
                      "WhitePop", "BlackPop", "NativeAmericanPop", "AsianPop", "NativeHawaiianPop",
                      "HispanicPop", "TwoOrMoreRacesPop"]]

census_18.head()

,State,Name,Population,PovertyCount,PovertyRate,UnemploymentRate,NoCompletedEducation,PopulationWithHighSchoolDiploma,PopulationWithAssociatesDegree,PopulationWithBachelorsDegree,WhitePop,BlackPop,NativeAmericanPop,AsianPop,NativeHawaiianPop,HispanicPop,TwoOrMoreRacesPop
0,28,Mississippi,2988762.0,599795.0,20.068343,3.681658,25771.0,474505.0,184334.0,266274.0,1751193.0,1125834.0,13689.0,28313.0,707.0,90493.0,40193.0
1,29,Missouri,6090062.0,837930.0,13.758973,2.560861,33534.0,1076664.0,323751.0,729543.0,5008342.0,704896.0,26952.0,116720.0,6662.0,249105.0,155328.0
2,30,Montana,1041732.0,139063.0,13.349211,2.153433,3151.0,175345.0,64097.0,148624.0,925642.0,4630.0,67289.0,7947.0,757.0,39019.0,29435.0
3,31,Nebraska,1904760.0,213790.0,11.223986,1.918614,14848.0,283361.0,130043.0,258291.0,1666463.0,90860.0,17282.0,44105.0,1392.0,203281.0,48600.0
4,32,Nevada,2922849.0,393431.0,13.460531,3.481124,32255.0,483098.0,163699.0,317510.0,1935103.0,261123.0,35845.0,234693.0,19352.0,831597.0,140499.0


In [6]:
#2017
c17 = Census(api_key, year=2017)

census_data17 = c17.acs5.get(("NAME", "B01003_001E",
                                      "B17001_002E",
                                      "B23025_005E",
                                      "B15003_002E",
                                      "B15003_017E",
                                      "B15003_021E",
                                      "B15003_022E",
                                      "B02001_002E",
                                      "B02001_003E",
                                      "B02001_004E",
                                      "B02001_005E",
                                      "B02001_006E",
                                      "B03001_003E",
                                      "B02001_008E"), {'for': 'state:*'})

# Convert to DataFrame
census_17 = pd.DataFrame(census_data18)

# Column Reordering
census_17 = census_17.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "PovertyCount",
                                      "B23025_005E": "UnemploymentCount",
                                      "B15003_002E": "NoCompletedEducation",
                                      "B15003_017E": "PopulationWithHighSchoolDiploma",
                                      "B15003_021E": "PopulationWithAssociatesDegree",
                                      "B15003_022E": "PopulationWithBachelorsDegree",
                                      "B02001_002E": "WhitePop",
                                      "B02001_003E": "BlackPop",
                                      "B02001_004E": "NativeAmericanPop",
                                      "B02001_005E": "AsianPop",
                                      "B02001_006E": "NativeHawaiianPop",
                                      "B03001_003E": "HispanicPop",
                                      "B02001_008E": "TwoOrMoreRacesPop",
                                      "NAME": "Name", "state": "State"})

# Add in Poverty Rate (Poverty Count / Population)
census_17["PovertyRate"] = 100 * \
    census_17["PovertyCount"].astype(
        int) / census_17["Population"].astype(int)
# Add in unemployment Rate (Employment Count / Population)
census_17["UnemploymentRate"] = 100 * \
    census_17["UnemploymentCount"].astype(
        int) / census_17["Population"].astype(int)
# Final DataFrame
census_17 = census_17[["State", "Name", "Population", "PovertyCount", "PovertyRate", "UnemploymentRate",
                        "NoCompletedEducation", "PopulationWithHighSchoolDiploma",
                      "PopulationWithAssociatesDegree", "PopulationWithBachelorsDegree",
                      "WhitePop", "BlackPop", "NativeAmericanPop", "AsianPop", "NativeHawaiianPop",
                      "HispanicPop", "TwoOrMoreRacesPop"]]

census_17.head()

,State,Name,Population,PovertyCount,PovertyRate,UnemploymentRate,NoCompletedEducation,PopulationWithHighSchoolDiploma,PopulationWithAssociatesDegree,PopulationWithBachelorsDegree,WhitePop,BlackPop,NativeAmericanPop,AsianPop,NativeHawaiianPop,HispanicPop,TwoOrMoreRacesPop
0,28,Mississippi,2988762.0,599795.0,20.068343,3.681658,25771.0,474505.0,184334.0,266274.0,1751193.0,1125834.0,13689.0,28313.0,707.0,90493.0,40193.0
1,29,Missouri,6090062.0,837930.0,13.758973,2.560861,33534.0,1076664.0,323751.0,729543.0,5008342.0,704896.0,26952.0,116720.0,6662.0,249105.0,155328.0
2,30,Montana,1041732.0,139063.0,13.349211,2.153433,3151.0,175345.0,64097.0,148624.0,925642.0,4630.0,67289.0,7947.0,757.0,39019.0,29435.0
3,31,Nebraska,1904760.0,213790.0,11.223986,1.918614,14848.0,283361.0,130043.0,258291.0,1666463.0,90860.0,17282.0,44105.0,1392.0,203281.0,48600.0
4,32,Nevada,2922849.0,393431.0,13.460531,3.481124,32255.0,483098.0,163699.0,317510.0,1935103.0,261123.0,35845.0,234693.0,19352.0,831597.0,140499.0


In [7]:
#2016
c16 = Census(api_key, year=2016)
census_data16 = c16.acs5.get(("NAME", "B01003_001E",
                                      "B17001_002E",
                                      "B23025_005E",
                                      "B15003_002E",
                                      "B15003_017E",
                                      "B15003_021E",
                                      "B15003_022E",
                                      "B02001_002E",
                                      "B02001_003E",
                                      "B02001_004E",
                                      "B02001_005E",
                                      "B02001_006E",
                                      "B03001_003E",
                                      "B02001_008E"), {'for': 'state:*'})

# Convert to DataFrame
census_16 = pd.DataFrame(census_data16)

# Column Reordering
census_16 = census_16.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "PovertyCount",
                                      "B23025_005E": "UnemploymentCount",
                                      "B15003_002E": "NoCompletedEducation",
                                      "B15003_017E": "PopulationWithHighSchoolDiploma",
                                      "B15003_021E": "PopulationWithAssociatesDegree",
                                      "B15003_022E": "PopulationWithBachelorsDegree",
                                      "B02001_002E": "WhitePop",
                                      "B02001_003E": "BlackPop",
                                      "B02001_004E": "NativeAmericanPop",
                                      "B02001_005E": "AsianPop",
                                      "B02001_006E": "NativeHawaiianPop",
                                      "B03001_003E": "HispanicPop",
                                      "B02001_008E": "TwoOrMoreRacesPop",
                                      "NAME": "Name", "state": "State"})

# Add in Poverty Rate (Poverty Count / Population)
census_16["PovertyRate"] = 100 * \
    census_16["PovertyCount"].astype(
        int) / census_16["Population"].astype(int)
# Add in Unemployment Rate (Employment Count / Population)
census_16["UnemploymentRate"] = 100 * \
    census_16["UnemploymentCount"].astype(
        int) / census_16["Population"].astype(int)
# Final DataFrame
census_16 = census_16[["State", "Name", "Population", "PovertyCount", "PovertyRate", "UnemploymentRate",
                        "NoCompletedEducation", "PopulationWithHighSchoolDiploma",
                      "PopulationWithAssociatesDegree", "PopulationWithBachelorsDegree",
                      "WhitePop", "BlackPop", "NativeAmericanPop", "AsianPop", "NativeHawaiianPop",
                      "HispanicPop", "TwoOrMoreRacesPop"]]

census_16.head()

,State,Name,Population,PovertyCount,PovertyRate,UnemploymentRate,NoCompletedEducation,PopulationWithHighSchoolDiploma,PopulationWithAssociatesDegree,PopulationWithBachelorsDegree,WhitePop,BlackPop,NativeAmericanPop,AsianPop,NativeHawaiianPop,HispanicPop,TwoOrMoreRacesPop
0,28,Mississippi,2989192.0,645553.0,21.596237,4.310195,26988.0,472088.0,170289.0,255615.0,1764038.0,1121327.0,13072.0,28562.0,492.0,86704.0,35367.0
1,29,Missouri,6059651.0,897755.0,14.815292,3.302880,33236.0,1079050.0,307566.0,701863.0,5000875.0,701896.0,25641.0,107953.0,6479.0,237284.0,148041.0
2,30,Montana,1023391.0,148677.0,14.527878,2.871043,2919.0,175084.0,60424.0,140945.0,911907.0,4260.0,67222.0,7481.0,841.0,34841.0,26522.0
3,31,Nebraska,1881259.0,227021.0,12.067504,2.261464,13559.0,288518.0,123619.0,245288.0,1655708.0,88388.0,15739.0,39794.0,1305.0,191802.0,43653.0
4,32,Nevada,2839172.0,417257.0,14.696433,4.702956,30124.0,466140.0,152171.0,293579.0,1933057.0,243552.0,31927.0,222612.0,18334.0,790497.0,129036.0


In [8]:
#2015
c15 = Census(api_key, year=2015)

census_data15 = c15.acs5.get(("NAME", "B01003_001E",
                                      "B17001_002E",
                                      "B23025_005E",
                                      "B15003_002E",
                                      "B15003_017E",
                                      "B15003_021E",
                                      "B15003_022E",
                                      "B02001_002E",
                                      "B02001_003E",
                                      "B02001_004E",
                                      "B02001_005E",
                                      "B02001_006E",
                                      "B03001_003E",
                                      "B02001_008E"), {'for': 'state:*'})

# Convert to DataFrame
census_15 = pd.DataFrame(census_data15)

# Column Reordering
census_15 = census_15.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "PovertyCount",
                                      "B23025_005E": "UnemploymentCount",
                                      "B15003_002E": "NoCompletedEducation",
                                      "B15003_017E": "PopulationWithHighSchoolDiploma",
                                      "B15003_021E": "PopulationWithAssociatesDegree",
                                      "B15003_022E": "PopulationWithBachelorsDegree",
                                      "B02001_002E": "WhitePop",
                                      "B02001_003E": "BlackPop",
                                      "B02001_004E": "NativeAmericanPop",
                                      "B02001_005E": "AsianPop",
                                      "B02001_006E": "NativeHawaiianPop",
                                      "B03001_003E": "HispanicPop",
                                      "B02001_008E": "TwoOrMoreRacesPop",
                                      "NAME": "Name", "state": "State"})

# Add in Poverty Rate (Poverty Count / Population)
census_15["PovertyRate"] = 100 * \
    census_15["PovertyCount"].astype(
        int) / census_15["Population"].astype(int)
# Add in Unemployment Rate (Employment Count / Population)
census_15["UnemploymentRate"] = 100 * \
    census_15["UnemploymentCount"].astype(
        int) / census_15["Population"].astype(int)
# Final DataFrame
census_15 = census_15[["State", "Name", "Population", "PovertyCount", "PovertyRate", "UnemploymentRate",
                        "NoCompletedEducation", "PopulationWithHighSchoolDiploma",
                      "PopulationWithAssociatesDegree", "PopulationWithBachelorsDegree",
                      "WhitePop", "BlackPop", "NativeAmericanPop", "AsianPop", "NativeHawaiianPop",
                      "HispanicPop", "TwoOrMoreRacesPop"]]

census_15.head()

,State,Name,Population,PovertyCount,PovertyRate,UnemploymentRate,NoCompletedEducation,PopulationWithHighSchoolDiploma,PopulationWithAssociatesDegree,PopulationWithBachelorsDegree,WhitePop,BlackPop,NativeAmericanPop,AsianPop,NativeHawaiianPop,HispanicPop,TwoOrMoreRacesPop
0,28,Mississippi,2988081.0,651545.0,21.804797,4.633241,27994.0,469777.0,166725.0,251230.0,1768174.0,1116457.0,12896.0,28900.0,409.0,86507.0,35334.0
1,29,Missouri,6045448.0,915972.0,15.151433,3.747084,32874.0,1079150.0,299637.0,685454.0,4995527.0,697022.0,24951.0,106537.0,6161.0,233046.0,147645.0
2,30,Montana,1014699.0,150586.0,14.840460,3.189222,2786.0,175079.0,57894.0,138099.0,904977.0,4757.0,65693.0,7187.0,848.0,33622.0,25777.0
3,31,Nebraska,1869365.0,231115.0,12.363289,2.546426,13074.0,292881.0,120763.0,238158.0,1647551.0,88195.0,16319.0,38066.0,1280.0,186268.0,41595.0
4,32,Nevada,2798636.0,426337.0,15.233742,5.304977,28887.0,457309.0,147840.0,285510.0,1929661.0,235721.0,31393.0,214846.0,17942.0,770553.0,122935.0


#### Add Year Column to each census dataframe

In [9]:
census_15["Year"] = "2015"
census_16["Year"] = "2016"
census_17["Year"] = "2017"
census_18["Year"] = "2018"
census_19["Year"] = "2019"

#### Combine into single census dataframe

In [10]:
census_combined = census_15.append(census_16)
census_combined = census_combined.append(census_17)
census_combined = census_combined.append(census_18)
census_combined = census_combined.append(census_19)
census_combined.head()

,State,Name,Population,PovertyCount,PovertyRate,UnemploymentRate,NoCompletedEducation,PopulationWithHighSchoolDiploma,PopulationWithAssociatesDegree,PopulationWithBachelorsDegree,WhitePop,BlackPop,NativeAmericanPop,AsianPop,NativeHawaiianPop,HispanicPop,TwoOrMoreRacesPop,Year
0,28,Mississippi,2988081.0,651545.0,21.804797,4.633241,27994.0,469777.0,166725.0,251230.0,1768174.0,1116457.0,12896.0,28900.0,409.0,86507.0,35334.0,2015
1,29,Missouri,6045448.0,915972.0,15.151433,3.747084,32874.0,1079150.0,299637.0,685454.0,4995527.0,697022.0,24951.0,106537.0,6161.0,233046.0,147645.0,2015
2,30,Montana,1014699.0,150586.0,14.840460,3.189222,2786.0,175079.0,57894.0,138099.0,904977.0,4757.0,65693.0,7187.0,848.0,33622.0,25777.0,2015
3,31,Nebraska,1869365.0,231115.0,12.363289,2.546426,13074.0,292881.0,120763.0,238158.0,1647551.0,88195.0,16319.0,38066.0,1280.0,186268.0,41595.0,2015
4,32,Nevada,2798636.0,426337.0,15.233742,5.304977,28887.0,457309.0,147840.0,285510.0,1929661.0,235721.0,31393.0,214846.0,17942.0,770553.0,122935.0,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,53,Washington,7404107.0,785244.0,10.605519,2.530082,59452.0,925707.0,509353.0,1144545.0,5581128.0,281683.0,94449.0,631333.0,49090.0,937579.0,433386.0,2019
48,54,West Virginia,1817305.0,310044.0,17.060647,2.856428,10076.0,432826.0,94488.0,159196.0,1691600.0,66990.0,3667.0,14523.0,419.0,28368.0,32135.0,2019
49,55,Wisconsin,5790716.0,639160.0,11.037668,1.926601,34775.0,1073887.0,429430.0,780824.0,4946961.0,371200.0,51392.0,161841.0,2400.0,394392.0,138572.0,2019
50,56,Wyoming,581024.0,62257.0,10.715048,2.331057,1881.0,93507.0,43761.0,68269.0,531304.0,5582.0,14164.0,5025.0,596.0,57341.0,15521.0,2019


#### Read in infant mortality csv

In [11]:
infant_mortality = pd.read_csv("data/2019mortality.csv", usecols=["YEAR", "STATE", "RATE", "DEATHS"])
infant_mortality.head()

,YEAR,STATE,RATE,DEATHS
0,2019,AL,7.89,449
1,2019,AK,4.81,48
2,2019,AZ,5.24,429
3,2019,AR,6.9,251
4,2019,CA,4.06,1879
...,...,...,...,...
345,2005,VA,7.47,781
346,2005,WA,5.07,419
347,2005,WV,8.16,170
348,2005,WI,6.54,464


#### Remove data for years 2005 and 2014

In [12]:
mortality_clean = infant_mortality[infant_mortality.YEAR != 2005]
mortality_clean = mortality_clean[mortality_clean.YEAR != 2014]
mortality_clean.head()

,YEAR,STATE,RATE,DEATHS
0,2019,AL,7.89,449
1,2019,AK,4.81,48
2,2019,AZ,5.24,429
3,2019,AR,6.9,251
4,2019,CA,4.06,1879
...,...,...,...,...
245,2015,VA,5.9,610
246,2015,WA,4.88,434
247,2015,WV,7.12,141
248,2015,WI,5.8,389


#### Read in CSV with state names and abbreviations. 

In [13]:
# csv from https://worldpopulationreview.com/states/state-abbreviations
states = pd.read_csv("Data/states.csv", usecols=["State", "Code"])
states.rename(columns={"State": "Name", "Code": "STATE"}, inplace=True)
states.head()

,Name,STATE
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


#### Merge state information dataframe with infant mortality dataframe

In [14]:
mortality_state = pd.merge(mortality_clean, states, on="STATE")

mortality_state.rename(columns={"YEAR": "Year", "RATE": "Rate", "DEATHS": "Deaths", "STATE": "State"}, inplace=True)
mortality_state.head()

,Year,State,Rate,Deaths,Name
0,2019,AL,7.89,449,Alabama
1,2018,AL,6.94,401,Alabama
2,2017,AL,7.38,435,Alabama
3,2016,AL,9.03,534,Alabama
4,2015,AL,8.31,496,Alabama


#### Merge dataframes into final daatframe with all census and infant mortality data

In [15]:
census_combined["Year"] = census_combined["Year"].astype(int)
# census_combined.info()
final_df = pd.merge(mortality_state, census_combined, how="inner", on=["Name", "Year"])
final_df

,Year,State_x,Rate,Deaths,Name,State_y,Population,PovertyCount,PovertyRate,UnemploymentRate,...,PopulationWithHighSchoolDiploma,PopulationWithAssociatesDegree,PopulationWithBachelorsDegree,WhitePop,BlackPop,NativeAmericanPop,AsianPop,NativeHawaiianPop,HispanicPop,TwoOrMoreRacesPop
0,2019,AL,7.89,449,Alabama,01,4876250.0,795989.0,16.323794,2.708946,...,841716.0,282316.0,529178.0,3320247.0,1299048.0,25565.0,66270.0,2238.0,208626.0,92220.0
1,2018,AL,6.94,401,Alabama,01,4864680.0,829400.0,17.049426,3.040241,...,838776.0,273493.0,515443.0,3317453.0,1293186.0,25576.0,64609.0,2182.0,203146.0,91619.0
2,2017,AL,7.38,435,Alabama,01,4864680.0,829400.0,17.049426,3.040241,...,838776.0,273493.0,515443.0,3317453.0,1293186.0,25576.0,64609.0,2182.0,203146.0,91619.0
3,2016,AL,9.03,534,Alabama,01,4841164.0,868666.0,17.943329,3.810633,...,828205.0,258502.0,492382.0,3325037.0,1282053.0,23919.0,60744.0,2008.0,193503.0,85412.0
4,2015,AL,8.31,496,Alabama,01,4830620.0,887260.0,18.367415,4.287172,...,826272.0,251335.0,478812.0,3325464.0,1276544.0,23850.0,59599.0,2439.0,193492.0,81646.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,2019,WY,7,46,Wyoming,56,581024.0,62257.0,10.715048,2.331057,...,93507.0,43761.0,68269.0,531304.0,5582.0,14164.0,5025.0,596.0,57341.0,15521.0
246,2018,WY,5.33,35,Wyoming,56,581836.0,63311.0,10.881245,2.352037,...,91927.0,42835.0,67697.0,532008.0,5540.0,14053.0,4756.0,539.0,56966.0,15579.0
247,2017,WY,4.64,32,Wyoming,56,581836.0,63311.0,10.881245,2.352037,...,91927.0,42835.0,67697.0,532008.0,5540.0,14053.0,4756.0,539.0,56966.0,15579.0
248,2016,WY,5.01,37,Wyoming,56,583029.0,65762.0,11.279370,2.593010,...,93152.0,41392.0,66753.0,531851.0,6435.0,13033.0,5385.0,391.0,56386.0,15531.0


#### Clean final dataframe

In [16]:
final_df.rename(columns={"State_x": "State"}, inplace=True)
final_df.drop(["State_y"], axis=1, inplace=True)
final_df

,Year,State,Rate,Deaths,Name,Population,PovertyCount,PovertyRate,UnemploymentRate,NoCompletedEducation,PopulationWithHighSchoolDiploma,PopulationWithAssociatesDegree,PopulationWithBachelorsDegree,WhitePop,BlackPop,NativeAmericanPop,AsianPop,NativeHawaiianPop,HispanicPop,TwoOrMoreRacesPop
0,2019,AL,7.89,449,Alabama,4876250.0,795989.0,16.323794,2.708946,40837.0,841716.0,282316.0,529178.0,3320247.0,1299048.0,25565.0,66270.0,2238.0,208626.0,92220.0
1,2018,AL,6.94,401,Alabama,4864680.0,829400.0,17.049426,3.040241,41240.0,838776.0,273493.0,515443.0,3317453.0,1293186.0,25576.0,64609.0,2182.0,203146.0,91619.0
2,2017,AL,7.38,435,Alabama,4864680.0,829400.0,17.049426,3.040241,41240.0,838776.0,273493.0,515443.0,3317453.0,1293186.0,25576.0,64609.0,2182.0,203146.0,91619.0
3,2016,AL,9.03,534,Alabama,4841164.0,868666.0,17.943329,3.810633,42577.0,828205.0,258502.0,492382.0,3325037.0,1282053.0,23919.0,60744.0,2008.0,193503.0,85412.0
4,2015,AL,8.31,496,Alabama,4830620.0,887260.0,18.367415,4.287172,41876.0,826272.0,251335.0,478812.0,3325464.0,1276544.0,23850.0,59599.0,2439.0,193492.0,81646.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,2019,WY,7,46,Wyoming,581024.0,62257.0,10.715048,2.331057,1881.0,93507.0,43761.0,68269.0,531304.0,5582.0,14164.0,5025.0,596.0,57341.0,15521.0
246,2018,WY,5.33,35,Wyoming,581836.0,63311.0,10.881245,2.352037,1519.0,91927.0,42835.0,67697.0,532008.0,5540.0,14053.0,4756.0,539.0,56966.0,15579.0
247,2017,WY,4.64,32,Wyoming,581836.0,63311.0,10.881245,2.352037,1519.0,91927.0,42835.0,67697.0,532008.0,5540.0,14053.0,4756.0,539.0,56966.0,15579.0
248,2016,WY,5.01,37,Wyoming,583029.0,65762.0,11.279370,2.593010,1102.0,93152.0,41392.0,66753.0,531851.0,6435.0,13033.0,5385.0,391.0,56386.0,15531.0


#### Write final dataframe to csv

In [17]:
final_df.to_csv("Data/infant_mortality.csv", encoding="utf-8", index=False)